In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
import gc
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten,Input,Activation,BatchNormalization,MaxPooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger
%load_ext tensorboard
import datetime
from tensorflow.keras.models import Model
from prettytable import PrettyTable
import prettytable
import warnings
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

warnings.filterwarnings("ignore")

In [3]:
def predict_class(class_index):
  if(class_index==0):
    return 'backyard'
  elif(class_index==1):
    return 'bathroom'
  elif(class_index==2):
    return 'bedroom'
  elif(class_index==3):
    return 'frontyard'
  elif(class_index==4):
    return 'kitchen'
  else:
     return 'livingRoom'

In [4]:
X_test=pd.read_pickle('/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_2/Pickle_Files/X_test_image_input_data_latest.pkl')

In [5]:
  # load json and create model
  json_file = open("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_2/Pickle_Files/EfficientNetB5_model.json", 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_2/Pickle_Files/EfficientNetB5_model.h5")

2021-09-16 05:56:25.587889: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-16 05:56:25.587944: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (613ab32cf086): /proc/driver/nvidia/version does not exist


In [38]:
%%writefile REIClassification.py
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten,Input,Activation,BatchNormalization,MaxPooling2D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger
import streamlit as st

st.title("Real Estate Image Classification")

X_test=pd.read_pickle('/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_2/Pickle_Files/X_test_image_input_data_latest.pkl')
json_file = open("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_2/Pickle_Files/EfficientNetB5_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_2/Pickle_Files/EfficientNetB5_model.h5")

def final_fun_1(selected_image='bedroom (419)'):
  image_data=X_test[X_test['image_name'].values==selected_image]['image_data'].values[0]
  pred_prob=loaded_model.predict(image_data)  
  return predict_class(np.argmax(pred_prob))

def predict_class(class_index):
  if(class_index==0):
    return 'backyard'
  elif(class_index==1):
    return 'bathroom'
  elif(class_index==2):
    return 'bedroom'
  elif(class_index==3):
    return 'frontyard'
  elif(class_index==4):
    return 'kitchen'
  else:
     return 'livingRoom'  



image_select = st.selectbox('Select Image: ',X_test['image_name'].values)
image_selection_msg=st.text('You have selected image: '+image_select)
st.image(X_test[X_test['image_name'].values==image_select]['image_data'].values[0])
if st.button('Predict Image Class'):
    image_predicted_class=final_fun_1(image_select)
    st.write('Image Name: '+image_select)
    st.write("Image Predicted Class: "+str(image_predicted_class))

Overwriting REIClassification.py


In [9]:
def final_func_2(image_data,image_class): 
    test_pred=[]
    for row in image_data:
        test_pred.append(predict_class(np.argmax(loaded_model.predict(row))))
    
    print("Accuracy: ",accuracy_score(image_class, test_pred))

In [11]:
final_func_2(X_test['image_data'],X_test['image_class'])

Accuracy:  0.9027303754266212


In [38]:
!pip install streamlit
#!pip install pyngrok
!pip install pyngrok==4.1.1

In [39]:
!ngrok authtoken 1x9kwOjrkR7yzIeRHqtYdnNdIrW_2gitL4cy88GMoTcqKnqK9

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [39]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

'http://c42c-34-90-220-75.ngrok.io'

In [40]:
!streamlit run --server.port 8501 REIClassification.py >/dev/null

2021-09-16 07:32:09.212821: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-16 07:32:09.212891: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (613ab32cf086): /proc/driver/nvidia/version does not exist
2021-09-16 07:34:01.464622: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
